In [ ]:
import pandas as pd
from openpyxl import load_workbook

def xlsx2df(filename):
    wb = load_workbook(filename, data_only=False)
    sheet = wb["Sheet1"]

    data = []
    for row in sheet.iter_rows():
        row_data = []
        for cell in row:
            row_data.append(cell.value)
        data.append(row_data)

    df = pd.DataFrame(data)
    return df

# df = xlsx2df("DCF model.xlsx")
df = xlsx2df("Actuarial Calculation.xlsx")
print(df)

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

model_openai = ChatOpenAI(model="gpt-4o", temperature=0, verbose=True)

agent = create_pandas_dataframe_agent(model_openai, df, allow_dangerous_code=True, number_of_head_rows=10)

In [ ]:
from langchain.globals import set_debug
set_debug(False)

response = agent.invoke("This dataframe contains cells from an Excel spreadsheet. Some of these cells contain data labels, some contain data and others contain formulas. Convert this dataframe to Python code such that data labels are variable names, data are variable values and formulas use the variable names. Convert all the cells and formulas. Note that the first row index in Excel is 1.")
print(response['output'])